<a href="https://colab.research.google.com/github/fininsight/text-mining-tutorial/blob/master/0_%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%A0%84%EC%B2%98%EB%A6%AC_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 전처리 (Text Preprocessing)

In [0]:
# pandas, numpy import
import pandas as pd
import numpy as np

# pandas, numpy 버젼 확인
print(pd.__version__)
print(np.__version__)

0.23.4
1.16.2


## 1 영문 토큰화
영문의 경우 띄어쓰기를 기준으로 단어를 구분지을 수 있기 때문에, 공백으로 토큰화가 가능하다.

NLTK lib (https://www.nltk.org/) 사용

### 1.1 실습용 영문기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

https://www.macworld.co.uk/news/apple/airpods-2-3672580/

In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
url = 'https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/?ss=ai-big-data#45dd5dd61f25'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [0]:
eng_news = soup.select('p[class="speakable-paragraph"]')
eng_text = eng_news[0].get_text()

In [0]:
eng_text

'It is the present-day darling of the tech world. The current renaissance of Artificial Intelligence (AI) with its sister discipline Machine Learning (ML) has led every IT firm worth its salt to engineer some form of AI onto its platform, into its toolsets and throughout its software applications.'

### 1.2 영문 토큰화
공백을 기준으로 토큰을 분리한다

In [0]:
import nltk
from nltk.tokenize import TreebankWordTokenizer

In [0]:
token = TreebankWordTokenizer().tokenize(eng_text)
print(token)

['It', 'is', 'the', 'present-day', 'darling', 'of', 'the', 'tech', 'world.', 'The', 'current', 'renaissance', 'of', 'Artificial', 'Intelligence', '(', 'AI', ')', 'with', 'its', 'sister', 'discipline', 'Machine', 'Learning', '(', 'ML', ')', 'has', 'led', 'every', 'IT', 'firm', 'worth', 'its', 'salt', 'to', 'engineer', 'some', 'form', 'of', 'AI', 'onto', 'its', 'platform', ',', 'into', 'its', 'toolsets', 'and', 'throughout', 'its', 'software', 'applications', '.']


### 1.3 영문 품사 부착 (PoS Tagging)
분리한 토큰마다 품사를 부착한다

https://www.nltk.org/book/ch05.html

In [0]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
taggedToken = pos_tag(token)
print(taggedToken)

[('It', 'PRP'), ('is', 'VBZ'), ('the', 'DT'), ('present-day', 'JJ'), ('darling', 'NN'), ('of', 'IN'), ('the', 'DT'), ('tech', 'NN'), ('world.', 'VBD'), ('The', 'DT'), ('current', 'JJ'), ('renaissance', 'NN'), ('of', 'IN'), ('Artificial', 'JJ'), ('Intelligence', 'NNP'), ('(', '('), ('AI', 'NNP'), (')', ')'), ('with', 'IN'), ('its', 'PRP$'), ('sister', 'NN'), ('discipline', 'NN'), ('Machine', 'NNP'), ('Learning', 'NNP'), ('(', '('), ('ML', 'NNP'), (')', ')'), ('has', 'VBZ'), ('led', 'VBN'), ('every', 'DT'), ('IT', 'NNP'), ('firm', 'NN'), ('worth', 'VBD'), ('its', 'PRP$'), ('salt', 'NN'), ('to', 'TO'), ('engineer', 'VB'), ('some', 'DT'), ('form', 'NN'), ('of', 'IN'), ('AI', 'NNP'), ('onto', 'IN'), ('its', 'PRP$'), ('platform', 'NN'), (',', ','), ('into', 'IN'), ('its', 'PRP$'), ('toolsets', 'NNS'), ('and', 'CC'), ('throughout', 'IN'), ('its', 'PRP$'), ('software', 'NN'), ('applications', 'NNS'), ('.', '.')]


### 1.4 개체명 인식 (NER, Named Entity Recognition)

In [0]:
nltk.download('words')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [0]:
from nltk import ne_chunk

In [0]:
neToken = ne_chunk(taggedToken)
print(neToken)

(S
  It/PRP
  is/VBZ
  the/DT
  present-day/JJ
  darling/NN
  of/IN
  the/DT
  tech/NN
  world./VBD
  The/DT
  current/JJ
  renaissance/NN
  of/IN
  (ORGANIZATION Artificial/JJ Intelligence/NNP)
  (/(
  AI/NNP
  )/)
  with/IN
  its/PRP$
  sister/NN
  discipline/NN
  (PERSON Machine/NNP Learning/NNP)
  (/(
  ML/NNP
  )/)
  has/VBZ
  led/VBN
  every/DT
  IT/NNP
  firm/NN
  worth/VBD
  its/PRP$
  salt/NN
  to/TO
  engineer/VB
  some/DT
  form/NN
  of/IN
  (ORGANIZATION AI/NNP)
  onto/IN
  its/PRP$
  platform/NN
  ,/,
  into/IN
  its/PRP$
  toolsets/NNS
  and/CC
  throughout/IN
  its/PRP$
  software/NN
  applications/NNS
  ./.)


#### 5) 원형 복원
각 토큰의 원형을 복원하여 표준화 한다. 

##### 어간추출 (Stemming)

In [0]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

print("running -> " + ps.stem("running"))
print("beautiful -> " + ps.stem("beautiful"))
print("believes -> " + ps.stem("believes"))
print("using -> " + ps.stem("using"))
print("conversation -> " + ps.stem("conversation"))
print("organization -> " + ps.stem("organization"))
print("studies -> " + ps.stem("studies"))

running -> run
beautiful -> beauti
believes -> believ
using -> use
conversation -> convers
organization -> organ
studies -> studi


In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

##### 표제어 추출 (Lemmatization)

In [0]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

print("running -> " + wl.lemmatize("running"))
print("beautiful -> " + wl.lemmatize("beautiful"))
print("believes -> " + wl.lemmatize("believes"))
print("using -> " + wl.lemmatize("using"))
print("conversation -> " + wl.lemmatize("conversation"))
print("organization -> " + wl.lemmatize("organization"))
print("studies -> " + wl.lemmatize("studies"))

running -> running
beautiful -> beautiful
believes -> belief
using -> using
conversation -> conversation
organization -> organization
studies -> study




---



## 2 한글 토큰화
영문은 공백으로 토큰화가 가능하지만, 한글의 경우 품사를 고려하여 토큰화 해야한다.

### 2.1 실습용 한글기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

http://news.chosun.com/site/data/html_dir/2018/07/10/2018071004121.html

In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
url = 'http://news.chosun.com/site/data/html_dir/2018/07/10/2018071004121.html'
response = requests.get(url)
response.encoding = 'utf-8'    # 한글이므로 encoding을 utf-8로 지정
soup = BeautifulSoup(response.text,'html.parser')

In [0]:
kor_news = soup.select('div[class="par"]')
kor_text = kor_news[0].get_text()

In [0]:
kor_text

'교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다. 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다. 하지만 지금은 모든 것이 달라졌다. 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다.스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다. 그는 2011년 아이패드 2를 공개하면서 "애플의 DNA는 기술만으로는 충분하지 않다. 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다. 이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다.실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다. 구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다. 메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다. 중국 알리바바그룹의 마윈 회장의 학부 전공은 영어였다.내가 속해 있는 하버드대·듀크대 연구팀은 미국 IT 기업 창업자들의 92%가 학사 학위를, 47%는 석사 학위 이상을 갖고 있음을 밝혀냈다. 창업자들의 세부 전공을 보면 37%만 공학·컴퓨터 기술이며, 수학 전공자는 2%뿐이었다. 이들의 전공은 경영·회계·보건·예술·인문학 등 매우 다양했다.컴퓨터 주변기기 제조 업체 로지텍의 브랙큰 대럴 CEO도 영문학을 전공했다. 최근 내가 그에게 "어떻게 5년여 만에 회사 주가를 450% 올릴 수 있었느냐"고 물었더니 그는 "우리 회사가 만드는 모든 제

### 2.2 한글 토큰화 및 형태소 분석

In [0]:
#konlpy 설치
!pip install konlpy

    100% |████████████████████████████████| 19.4MB 1.7MB/s 
    100% |████████████████████████████████| 174kB 31.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/0e/2b/e8/c0b818ac4b3d35104d35e48cdc7afe27fc06ea277feed2831a
Successfully built JPype1


In [0]:
#출력을 위해서 임시 class 
class List(list): 
    def __str__(self): 
        return "[" + ", ".join(["%s" % x for x in self]) + "]"

한글 자연어처리기 비교

https://blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221337575742

In [0]:
# 코모란(Komoran) 토큰화
from konlpy.tag import Komoran
komoran= Komoran()
komoran_tokens = komoran.morphs(kor_text)
print(List(komoran_tokens))

# 한나눔(Hannanum) 토큰화
from konlpy.tag import Hannanum
hannanum= Hannanum()
hannanum_tokens = hannanum.morphs(kor_text)
print(List(hannanum_tokens))

# Okt 토큰화
from konlpy.tag import Okt
okt= Okt()
okt_tokens = okt.morphs(kor_text)
print(List(okt_tokens))

# Kkma 토큰화
from konlpy.tag import Kkma
kkma= Kkma()
kkma_tokens = kkma.morphs(kor_text)
print(List(kkma_tokens))

[교육, 을, 삶, 의, 최, 우선, 순위, 로, 두, 고, 있, 는, 한국, 의, 부모, 들, 은, 대학, 전공, 가운데, 의학, 과, 공학, ·, 과학, 을, 중시, 하, ㄴ다, ., 자녀, 의, 직업, 적, 성공, 을, 위하, 아, 대학, 전공, 으로, 의학, 과, 이공, 계, 를, 우선, 적, 으로, 고려, 하, 는, 일, 은, 한국, 이, 산업화, 중, 이, 던, 상황, 에서, ㄴ, 올바르, ㄴ, 선택, 이, 었, 다, ., 하지만, 지금, 은, 모든, 것, 이, 달라지, 었, 다, ., 요즘, 실리콘밸리, 에서, 확인, 되, 는, 것, 은, 4, 차, 산업, 혁명, 시대, 에, 는, 예술, 과, 인문학, 이, 의학, ·, 공학, 만큼, 중요, 하, 다는, 사실, 이, 다, ., 스티브 잡스, 는, 자신, 이, 대학, 시절, 수강, 하, 았, 던, 서, 체, (, 書體, ), 수업, 이, 매킨토시, (, 애플, 이, 1984, 년, 발표, 하, ㄴ, 개인용 컴퓨터, ), 개발, 성공, 에, 크, ㄴ, 영향, 을, 미치, 었, 다고, 말, 하, 았, 다, ., 그, 는, 2011, 년, 아이패드 2, 를, 공개, 하, 면서, ", 애플, 의, DNA, 는, 기술, 만, 으로, 는, 충분, 하, 지, 않, 다, ., 교양, 과, 인문학, 이, 결합, 하, ㄴ, 기술, 이야말로, 가슴, 벅차, ㄴ, 결과, 를, 낳, 을, 것, ", 이, 라며, 예술, 과, 디자인, 의, 중요, 성, 을, 강조, 하, 았, 다, ., 이런, 관점, 을, 바탕, 으로, 잡스, 는, 세계, 최고, 가치, 를, 인정받, 는, 기업, 을, 만들, 었, 고, ,, 기술, 산업, 의, 새롭, ㄴ, 표준, 까지, 정하, 았, 다, ., 실리콘밸리, 에서, 최근, 뜨, 고, 있, 는, 스타, 기업인, 중, 에, 는, 인문학, 전공자, 들, 이, 제법, 많, 다, ., 구인, ·, 구직, 소, 시, 어, ㄹ, 네트, 워킹, 서비스, 기업인, 링크, 드, 인, (, LinkedIn, ), 창

### 2.3 한글 품사 부착 (PoS Tagging)

PoS Tag 목록

https://docs.google.com/spreadsheets/u/1/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [0]:
# 코모란(Komoran) 품사 태깅
komoranTag = []
for token in komoran_tokens:
    komoranTag += komoran.pos(token)
print(komoranTag)

# 한나눔(Hannanum) 품사 태깅
hannanumTag = []
for token in hannanum_tokens:
    hannanumTag += hannanum.pos(token)
print(hannanumTag)

# Okt 품사 태깅
oktTag = []
for token in okt_tokens:
    oktTag += okt.pos(token)
print(oktTag)

# Kkma 품사 태깅
kkmaTag = []
for token in kkma_tokens:
    kkmaTag += kkma.pos(token)
print(kkmaTag)

[('교육', 'NNG'), ('을', 'NNG'), ('삶', 'NNG'), ('의', 'NNG'), ('최', 'NNP'), ('우선', 'MAG'), ('순위', 'NNP'), ('로', 'NNG'), ('두', 'MM'), ('고', 'MM'), ('있', 'VV'), ('늘', 'VV'), ('ㄴ', 'ETM'), ('한국', 'NNP'), ('의', 'NNG'), ('부모', 'NNG'), ('들', 'VV'), ('ㄹ', 'ETM'), ('은', 'NNP'), ('대학', 'NNG'), ('전공', 'NNG'), ('가운데', 'NNG'), ('의학', 'NNP'), ('과', 'NNG'), ('공학', 'NNP'), ('·', 'SP'), ('과학', 'NNG'), ('을', 'NNG'), ('중시', 'NNG'), ('하', 'NNG'), ('ㄴ다', 'EC'), ('.', 'SF'), ('자녀', 'NNG'), ('의', 'NNG'), ('직업', 'NNG'), ('적', 'NNB'), ('성공', 'NNG'), ('을', 'NNG'), ('위하', 'VV'), ('아', 'IC'), ('대학', 'NNG'), ('전공', 'NNG'), ('으로', 'JKB'), ('의학', 'NNP'), ('과', 'NNG'), ('이공', 'NNP'), ('계', 'NNG'), ('를', 'JKO'), ('우선', 'MAG'), ('적', 'NNB'), ('으로', 'JKB'), ('고려', 'NNP'), ('하', 'NNG'), ('늘', 'VV'), ('ㄴ', 'ETM'), ('일', 'NNB'), ('은', 'NNP'), ('한국', 'NNP'), ('이', 'MM'), ('산업화', 'NNP'), ('중', 'NNB'), ('이', 'MM'), ('덜', 'VV'), ('ㄴ', 'ETM'), ('상황', 'NNG'), ('에서', 'JKB'), ('ㄴ', 'JX'), ('올', 'MM'), ('바르', 'NNP'), ('ㄴ', 'JX'), ('선택

### 2.4 불용어(Stopword) 처리
분석에 불필요한 품사를 제거하고, 불필요한 단어(불용어)를 제거한다

In [0]:
#불용어 처리
stopPos = ['Suffix','Punctuation','Josa','Foreign','Alpha','Number']

In [0]:
# 최빈어 조회. 최빈어를 조회하여 불용어 제거 대상을 선정
from collections import Counter
Counter(oktTag).most_common()

[(('.', 'Punctuation'), 35),
 (('을', 'Josa'), 32),
 (('의', 'Noun'), 24),
 (('이', 'Noun'), 20),
 (('·', 'Punctuation'), 18),
 (('를', 'Noun'), 15),
 (('는', 'Verb'), 15),
 (('들', 'Verb'), 13),
 (('은', 'Noun'), 11),
 (('에', 'Josa'), 11),
 (('가', 'Verb'), 11),
 (('기술', 'Noun'), 10),
 (("'", 'Punctuation'), 10),
 (('과', 'Noun'), 9),
 (('"', 'Punctuation'), 9),
 ((',', 'Punctuation'), 9),
 (('수', 'Noun'), 9),
 (('전공', 'Noun'), 8),
 (('한', 'Verb'), 7),
 (('했다', 'Verb'), 7),
 (('적', 'Noun'), 6),
 (('것', 'Noun'), 6),
 (('(', 'Punctuation'), 6),
 ((')', 'Punctuation'), 6),
 (('자', 'Noun'), 6),
 (('공학', 'Noun'), 5),
 (('인문학', 'Noun'), 5),
 (('디자인', 'Noun'), 5),
 (('와', 'Verb'), 5),
 (('우리', 'Noun'), 5),
 (('로', 'Noun'), 4),
 (('있는', 'Adjective'), 4),
 (('한다', 'Verb'), 4),
 (('하는', 'Verb'), 4),
 (('일', 'Noun'), 4),
 (('예술', 'Noun'), 4),
 (('이다', 'Josa'), 4),
 (('컴퓨터', 'Noun'), 4),
 (('그', 'Noun'), 4),
 (('전공자', 'Noun'), 4),
 (('창업', 'Noun'), 4),
 (('하고', 'Verb'), 4),
 (('있다', 'Adjective'), 4),
 (('

In [0]:
stopWord = ['의','이','를','은','과','수','했다','것','있는','한다','하는','그','있다','할','이런','되기','해야','있게','여기']

In [0]:
word = []
for tag in oktTag:
    if tag[1] not in stopPos:
        if tag[0] not in stopWord:
            word.append(tag[0])

In [0]:
print(word)

['교육', '삶', '최', '우선', '순위', '로', '두고', '한국', '부모', '들', '대학', '전공', '가운데', '의학과', '공학', '과학', '중시', '자녀', '직업', '적', '성공', '위해', '대학', '전공', '의학과', '이공', '계', '우선', '적', '고려', '일', '한국', '산업화', '중', '상황', '올바른', '선택', '이었다', '하지만', '지금', '모든', '달라졌다', '요즘', '실리콘밸리', '확인', '되는', '차', '산업혁명', '시대', '예술', '인문학', '의학', '공학', '만큼', '중요하다는', '사실', '스티브', '잡스', '는', '자신', '대학', '시절', '수강', '했던', '서체', '수업', '매킨토시', '애플', '발표', '한', '개인', '용', '컴퓨터', '개발', '성공', '큰', '영향', '미쳤다고', '말', '는', '아이패드', '공개', '하면서', '애플', '는', '기술', '충분하지', '않다', '교양', '인문학', '결합', '한', '기술', '가슴', '벅찬', '결과', '낳을', '라며', '예술', '디자인', '중요성', '강조', '관점', '바탕', '잡스', '는', '세계', '최고', '가치', '인정받는', '기업', '만들었고', '기술', '산업', '새로운', '표준', '정', '실리콘밸리', '최근', '뜨고', '스타', '기업인', '중', '인문학', '전공자', '들', '제법', '많다', '구인', '구직', '소셜', '네트워킹', '서비스', '기업인', '링크드인', '창업', '자', '리드', '호프', '철학', '석사', '학위', '소지', '자', '잔', '보이', '치키', '유튜브', '는', '역사', '와', '문학', '전공', '메신저', '개발', '업체', '슬랙', '창업', '자', '스튜어트', '버터', '필드', '는